![Degirum banner](https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/degirum_banner.png)
## Multi-Source and Multi-Model AI Inference
This notebook is an example of how to perform AI inferences of multiple models processing multiple video streams.
Each video stream is fed to every model. Each model processes frames from every video stream in a multiplexing manner.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `hw_location` option. 

When running this notebook locally, you need to specify your cloud API access token in the [env.ini](../../env.ini) file, located in the same directory as this notebook.

When running this notebook in Google Colab, the cloud API access token should be stored in a user secret named `DEGIRUM_CLOUD_TOKEN`.

The script can use a web camera(s) or local camera(s) connected to the machine running this code or it can use video file(s).
The camera index or URL or video file path should be specified in the code below by assigning `video_sources`.

In [ ]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

#### Specify video sources and AI model names here

In [1]:
# hw_location: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local machine
#     IP address for AI server inference
# video_sources: list of video sources
#     camera index for local camera
#     URL of RTSP stream
#     URL of YouTube Video
#     path to video file (mp4 etc)
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# model_names: list of AI models to use for inferences (NOTE: they should have the same input size)
hw_location = "@cloud"
video_sources = [
    "https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/WalkingPeople.mp4",
    "https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/Traffic.mp4",
]
model_zoo_url = "degirum/public"
model_names = [
    "yolo_v5s_hand_det--512x512_quant_n2x_orca1_1",
    "yolo_v5s_face_det--512x512_quant_n2x_orca1_1",
    "yolo_v5n_car_det--512x512_quant_n2x_orca1_1",
    "yolo_v5s_person_det--512x512_quant_n2x_orca1_1",
]

#### The rest of the cells below should run without any modifications

In [ ]:
import degirum as dg, degirum_tools
from degirum_tools import streams as dgstreams

# create PySDK AI model objects
models = [
    dg.load_model(
        model_name=model_name,
        inference_host_address=hw_location,
        zoo_url=model_zoo_url,
        token=degirum_tools.get_token(),
        overlay_line_width=2,
    )
    for model_name in model_names
]

# check that all models have the same input configuration
assert all(
    type(model._preprocessor) == type(models[0]._preprocessor)
    and model.model_info.InputH == models[0].model_info.InputH
    and model.model_info.InputW == models[0].model_info.InputW
    for model in models[1:]
)

# create video source gizmos;
# stop_composition_on_end=True to stop whole composition when one (shorter) video source ends
sources = [
    dgstreams.VideoSourceGizmo(src, stop_composition_on_end=True)
    for src in video_sources
]

# create image resizer gizmos, one per video source
# (we use separate resizers to do resize only once per source to improve performance)
resizers = [dgstreams.AiPreprocessGizmo(models[0]) for _ in video_sources]

# create multi-input detector gizmos, one per model
detectors = [
    dgstreams.AiSimpleGizmo(model, inp_cnt=len(video_sources)) for model in models
]

# create result combiner gizmo to combine results from all detectors into single result
combiner = dgstreams.AiResultCombiningGizmo(len(models))

# create multi-window video multiplexing display gizmo
win_captions = [f"Stream #{i}: {str(src)}" for i, src in enumerate(video_sources)]
display = dgstreams.VideoDisplayGizmo(
    win_captions, show_ai_overlay=True, show_fps=True, multiplex=True
)

# connect all gizmos in the pipeline
# source[i] -> resizer[i] -> detector[j] -> combiner -> display
pipeline = (
    # each source is connected to corresponding resizer
    (source >> resizer for source, resizer in zip(sources, resizers)),
    # each resizer is connected to every detector
    (
        resizer >> detector[ri]
        for detector in detectors
        for ri, resizer in enumerate(resizers)
    ),
    # each detector is connected to result combiner
    (detector >> combiner[di] for di, detector in enumerate(detectors)),
    # result combiner is connected to display
    combiner >> display,
)

# create and start composition with given pipeline
dgstreams.Composition(*pipeline).start()